In [2]:
import numpy as np
import pandas as pd
import requests, os
import time
import random
# import csv
# import datetime as dt
# import json
# import os
# import statistics

In [4]:
def toggle_vpn():
    global vpn_on
    if vpn_on: 
        os.system("pkill -x NordVPN\ IKE")
        print('Turning OFF')
    else:
        os.system("open /Applications/NordVPN\ IKE.app")
        print('Turning ON')
    time.sleep(10)
    vpn_on = not vpn_on
    print("Toggled VPN")

In [320]:
os.system("pkill -x openvpn")

256

In [3]:
# vpn_on = True
# os.system("open /Applications/NordVPN\ IKE.app")
from fake_useragent import UserAgent
ua = UserAgent()

user_agent = {'User-agent': ua.random}

def get_request(url, parameters=None, rate=60): 
    response = None
    while not response:
        try:
            response = requests.get(url=url, params=parameters, headers=user_agent)
            if not response:
                print('None response. Trying again in {} seconds...'.format(rate))
#                 toggle_vpn()
                time.sleep(rate)
        except:
            print('Request Exception. Trying again in 5 sec...')
            time.sleep(5)
            continue
        
    
    return response.json()

Error occurred during loading data. Trying to use cache server https://fake-useragent.herokuapp.com/browsers/0.1.11
Traceback (most recent call last):
  File "/Users/matteofortier/opt/anaconda3/lib/python3.8/site-packages/fake_useragent/utils.py", line 154, in load
    for item in get_browsers(verify_ssl=verify_ssl):
  File "/Users/matteofortier/opt/anaconda3/lib/python3.8/site-packages/fake_useragent/utils.py", line 99, in get_browsers
    html = html.split('<table class="w3-table-all notranslate">')[1]
IndexError: list index out of range


In [ ]:
url = "https://steamspy.com/api.php"

for i in range(0,45):
    parameters = {"request": "all", "page": i}
    print("\'request\' = \'all\', 'page' = {}".format(i))
    time.sleep(60)
    json_data = get_request(url, parameters=parameters)
    app_list = pd.DataFrame.from_dict(json_data, orient='index')

    if not os.path.isfile('data/raw/app_list.csv'):
        app_list.to_csv('data/raw/app_list.csv', index=False)
    else: 
        app_list.to_csv('data/raw/app_list.csv', mode='a', header=False, index=False)



In [323]:
app_list.size

44854

In [ ]:
app_list = pd.read_csv('data/raw/app_list.csv', usecols=['appid'])['appid']
url = "https://steamspy.com/api.php"

for i in range(,app_list.size):
    parameters = {"request": "appdetails", "appid": app_list[i]}
    print(i)
    time.sleep(0.5)
    json_data = get_request(url, parameters=parameters, rate=1)
    try:
        json_data['tag_values'] = ', '.join(map(str, list(json_data['tags'].values())))
        json_data['tags'] = ', '.join(map(str, list(json_data['tags'].keys())))
    except:
        json_data['tags'] = ''
        json_data['tag_values'] = ''

        
    app_data = pd.DataFrame(json_data, index=[i])
    if not os.path.isfile('data/raw/app_data.csv'):
        app_data.to_csv('data/raw/app_data.csv', index=True)
    else: 
        app_data.to_csv('data/raw/app_data.csv', mode='a', header=False, index=True)
        

In [1]:
appid = app_list[44266]
parameters = {"appids": appid}
json_data = get_request(url, parameters=parameters, rate=2)
json_data

NameError: name 'app_list' is not defined

In [6]:
app_list = pd.read_csv('data/raw/app_list.csv', usecols=['appid'])['appid']
url = "http://store.steampowered.com/api/appdetails/"

for i in range(228,229):
    appid = app_list[i]
    parameters = {"appids": appid}
    print(i)
#     time.sleep(0.5)
    
    json_data = get_request(url, parameters=parameters, rate=10)[str(appid)]


    if json_data['success']:
        json_data = json_data['data']
    else:
        json_data = {'steam_appid': appid}
    
    all_keys = set(json_data.keys())
    desired_keys = ['type','name','steam_appid','required_age','is_free','controller_support','supported_languages','developers','publishers','platforms','metacritic','categories','genres','recommendations','release_date']
    
    missing_keys = set(desired_keys).difference(all_keys)
    for key in missing_keys:
        try:
            json_data[key] = None
        except:
            continue

    try:
        json_data['categories'] = ', '.join([x['description'] for x in json_data['categories']])
    except:
        json_data['categories'] = ''

    try:
        json_data['genres'] = ', '.join([x['description'] for x in json_data['genres']])
    except:
        json_data['genres'] = ''

    try:
        json_data['platforms'] = ', '.join([x[0] for x in json_data['platforms'].items() if x[1]])
    except:
        json_data['platforms'] = ''

    try:
        json_data['metacritic'] = json_data['metacritic']['score']
    except:
        json_data['metacritic'] = np.nan
    
    try:
        json_data['recommendations'] = json_data['recommendations']['total']
    except:
        json_data['recommendations'] = np.nan
        
    try:
        json_data['release_date'] = json_data['release_date']['date']
    except:
        try:
            if json_data['release_date']['coming_soon'] == True:
                json_data['release_date'] = 'coming soon'
            else:
                json_data['release_date'] = ''
        except:
            json_data['release_date'] = ''
    
    try:
        json_data['developers'] = ', '.join(json_data['developers'])
    except:
        json_data['developers'] = ''

    try:
        json_data['publishers'] = ', '.join(json_data['publishers'])
    except:
        json_data['publishers'] = ''

    json_data = {k: json_data[k] for k in desired_keys}    
    store_data = pd.DataFrame(json_data, index=[i])
    if not os.path.isfile('data/raw/store_data.csv'):
        store_data.to_csv('data/raw/store_data.csv', index=True)
    else:
        store_data.to_csv('data/raw/store_data.csv', mode='a', header=False, index=True)

44666
44667
44668
44669
44670
44671
44672
44673
44674
44675
44676
44677
44678
44679
44680
44681
44682
44683
44684
44685
44686
44687
44688
44689
44690
44691
44692
44693
44694
44695
44696
44697
44698
44699
44700
44701
44702
44703
44704
44705
44706
44707
44708
44709
44710
44711
44712
44713
44714
44715
44716
44717
44718
44719
44720
44721
44722
44723
44724
44725
44726
44727
44728
44729
44730
44731
44732
44733
44734
44735
44736
44737
44738
44739
44740
44741
44742
44743
44744
44745
44746
44747
44748
44749
44750
44751
44752
44753
44754
44755
44756
44757
44758
44759
44760
44761
44762
44763
44764
44765
44766
44767
44768
44769
44770
44771
44772
44773
44774
44775
44776
44777
44778
44779
44780
44781
44782
44783
44784
44785
44786
44787
44788
44789
44790
44791
44792
44793
44794
44795
44796
44797
44798
44799
44800
44801
44802
44803
44804
44805
44806
44807
44808
44809
44810
44811
44812
44813
44814
44815
44816
44817
44818
44819
44820
44821
44822
44823
44824
44825
44826
44827
44828
44829
44830
44831
4483

In [20]:
app_list = pd.read_csv('data/raw/app_list.csv', usecols=['appid'])['appid']
url = "http://store.steampowered.com/api/appdetails/"

i=43735

appid = app_list[i]
parameters = {"appids": appid}
print(appid)
#     time.sleep(0.5)

json_data = get_request(url, parameters=parameters, rate=10)[str(appid)]


if json_data['success']:
    json_data = json_data['data']
else:
    json_data = {'steam_appid': appid}

all_keys = set(json_data.keys())
desired_keys = ['type','name','steam_appid','required_age','is_free','controller_support','supported_languages','developers','publishers','platforms','metacritic','categories','genres','recommendations','release_date']

missing_keys = set(desired_keys).difference(all_keys)
for key in missing_keys:
    try:
        json_data[key] = None
    except:
        continue

try:
    json_data['categories'] = ', '.join([x['description'] for x in json_data['categories']])
except:
    json_data['categories'] = ''

try:
    json_data['genres'] = ', '.join([x['description'] for x in json_data['genres']])
except:
    json_data['genres'] = ''

try:
    json_data['platforms'] = ', '.join([x[0] for x in json_data['platforms'].items() if x[1]])
except:
    json_data['platforms'] = ''

try:
    json_data['metacritic'] = json_data['metacritic']['score']
except:
    json_data['metacritic'] = np.nan

try:
    json_data['recommendations'] = json_data['recommendations']['total']
except:
    json_data['recommendations'] = np.nan

try:
    json_data['release_date'] = json_data['release_date']['date']
except:
    try:
        if json_data['release_date']['coming_soon'] == True:
            json_data['release_date'] = 'coming soon'
        else:
            json_data['release_date'] = ''
    except:
        json_data['release_date'] = ''

try:
    json_data['developers'] = ', '.join(json_data['developers'])
except:
    json_data['developers'] = ''

try:
    json_data['publishers'] = ', '.join(json_data['publishers'])
except:
    json_data['publishers'] = ''

json_data = {k: json_data[k] for k in desired_keys}    
#     print(json_data)
store_data = pd.DataFrame(json_data, index=[i])
if not os.path.isfile('data/raw/store_data_english_fixes.csv'):
    store_data.to_csv('data/raw/store_data_english_fixes.csv', index=True)
else:
    store_data.to_csv('data/raw/store_data_english_fixes.csv', mode='a', header=False, index=True)

1434230
